In [42]:
from collections import Counter
from functools import reduce
from operator import add
from random import choice
from random import choice
import jieba
import pandas as pd
import random
import re

In [3]:
teacher="""
requirement = 对象 需要 寻找 数量 方法 解决 问题
对象 = 你 | 你们 | 他| 她 | 他们
需要 = 必须 | 一定要 | 需要
寻找 = 找到 | 发现 | 知晓 | 明白
数量 = 数字 个
数字 = 单个数字 | 数字 单个数字
单个数字 = 1 | 2 | 3 | 4 | 5 | 6
方法 = 办法 | 方案 | 论据 | 论点
解决 = 搞定 | 弄好 | 修理
问题 = 难题 | 题目 | 机器 | 车辆
"""

In [4]:
student = """
answer = 代词 谓语 宾语
代词 = 这 | 那
谓语 = 简直是 | 真的是 | 必须是
宾语 = 小菜一碟 | 难如登天 | 随手拈来
"""

In [ ]:
def adj_grammar()

In [5]:
def create_grammar(gram_str, split='='):
    grammar = {}
    for line in gram_str.split('\n'):
        if not line.strip():
            continue
        exp, stmt = line.split(split)
        grammar[exp.strip()] = [s.split() for s in stmt.split('|')]
    return grammar

In [73]:
def generate(gram, target):
    if target not in gram:
        return target
    expanded = [generate(gram, t) for t in choice(gram[target])]
    return ''.join([e for e in expanded if e != 'null'])

In [146]:
def generate_n(n):
    for i in range(n):
        yield generate(gram=create_grammar(teacher, split='='), target='requirement'), generate(gram=create_grammar(student, split='='), target='answer')

In [75]:
for line in generate_n(5):
    print(line)

('她必须找到6个方案修理机器', '那必须是小菜一碟')
('你们必须明白5个方案弄好机器', '那真的是难如登天')
('他一定要找到5611个方案修理机器', '这真的是小菜一碟')
('他一定要知晓34个方案搞定车辆', '那必须是小菜一碟')
('他们一定要明白2个论点修理机器', '那必须是小菜一碟')


In [49]:
filename = 'D:\\NLP\\material\\datasource-master\\sqlResult_1558436.csv'

In [50]:
content = pd.read_csv(filename, encoding='gb18030')

In [51]:
articles = content['content'].tolist()

In [52]:
def token(string):
    return re.findall('\w+', string)

In [54]:
articles_clean = [''.join(token(str(a))) for a in articles]

In [56]:
with open('D:\\NLP\\material\\datasource-master\\article_9k.txt', 'w') as f:
    for a in articles_clean:
        f.write(a + '\n')

In [57]:
def cut(string):
    return list(jieba.cut(string))

In [58]:
TOKEN = []

In [60]:
for i, line in enumerate((open('D:\\NLP\\material\\datasource-master\\article_9k.txt'))):
    TOKEN += cut(line)

In [61]:
words_count = Counter(TOKEN)

In [65]:
def prob_1(word):
    return words_count[word] / len(TOKENS)

In [66]:
TOKEN_2_GRAM = [''.join(TOKEN[i:i+2]) for i in range(len(TOKEN[:-2]))]

In [67]:
words_count_2 = Counter(TOKEN_2_GRAM)

In [164]:
def prob_2(word1, word2):
    if word1 + word2 in words_count_2: return words_count_2[word1+word2] / len(TOKEN_2_GRAM)
    else:
        return 1 / len(TOKEN_2_GRAM)

In [166]:
def get_probability(sentence):
    words = cut(sentence)
    sentence_pro = 1
    for i, word in enumerate(words[:-1]):
        next_ = words[i+1]
        probability = prob_2(word, next_)
        sentence_pro *= probability
    return sentence_pro

In [121]:
sen_prob = []

In [171]:
def generate_best(n):
    prob_dic = {}
    for sen in [generate(gram=create_grammar(teacher, split='='), target='requirement') for i in range(n)]:
        prob_dic[sen] = get_probability(sen)
    probabilities = [probability for probability in zip(prob_dic.keys(), prob_dic.values())]
    prob_sorted = sorted(probabilities, key=lambda x:x[1], reverse=True)
    return prob_sorted[0]

In [170]:
generate_best(5)

她必须知晓2251个办法弄好难题 1.1633789862471658e-56
你必须找到5162个方案修理机器 3.4161173585345166e-49
你必须找到46个办法弄好题目 3.3737990601167806e-54
他们一定要找到6个论点修理车辆 1.2809712659477205e-50
你们一定要找到3个论点弄好难题 1.4055845965831788e-57


('你必须找到5162个方案修理机器', 3.4161173585345166e-49)